# chroma analysis

In [1]:
import numpy as np
import scipy
import csv
import scipy.io as scio
import import_ipynb
import colour
from scipy.optimize import curve_fit

In [2]:
path = 'F:/code/'

In [3]:
def blackbody1931():
    B_file = path+'blackbody.csv'
    B_data = np.loadtxt(open(B_file),delimiter = ',')
    
    x_B = B_data[:,0]
    y_B = B_data[:,1]
    return x_B, y_B

In [4]:
def XYZ(WL_interp):
    XYZ_file = path+'color_xyz.csv'
    XYZ_data = np.loadtxt(open(XYZ_file),delimiter = ',')
    
    XYZ_WL = XYZ_data[:,0]
    X_lambda = XYZ_data[:,1]
    Y_lambda = XYZ_data[:,2]
    Z_lambda = XYZ_data[:,3]
    
    Xlambda_fun = scipy.interpolate.interp1d(XYZ_WL,X_lambda,kind='linear')
    X_lambda = Xlambda_fun(WL_interp)
    Ylambda_fun = scipy.interpolate.interp1d(XYZ_WL,Y_lambda,kind='linear')
    Y_lambda = Ylambda_fun(WL_interp)
    Zlambda_fun = scipy.interpolate.interp1d(XYZ_WL,Z_lambda,kind='linear')
    Z_lambda = Zlambda_fun(WL_interp)
    return(X_lambda,Y_lambda,Z_lambda)

In [5]:
def chroma_cordinate1931(p,x_lambda,y_lambda,z_lambda):
    k = 100/np.sum(p*y_lambda)
    X = k*np.sum(p*x_lambda)
    Y = k*np.sum(p*y_lambda)
    Z = k*np.sum(p*z_lambda)
    
    x = X/(X+Y+Z)
    y = Y/(X+Y+Z)
    
    return (x,y)

In [6]:
def CCT(x,y):
    n = (x-0.3290)/(y-0.1870)
    CCT = 669*n**4-779*n**3+3660*n**2-7047*n+5652
    return CCT

In [7]:
def trans_1931_1960(x,y):
    u = 4*x/(-2*x+12*y+3)
    v = 6*y/(-2*x+12*y+3)
    return (u,v)

In [8]:
def trans_1931_1976(x,y):
    u = 4*x/(-2*x+12*y+3)
    v = 9*y/(-2*x+12*y+3)
    return (u,v)

In [9]:
def rgb(p,x_lambda,y_lambda,z_lambda,WL_interp):
    def fun3(x,a1,a2,a3,m1,m2,m3,s1,s2,s3):
        return (a1*np.exp(-((x-m1)/s1)**2)+
                a2*np.exp(-((x-m2)/s2)**2)+
                a3*np.exp(-((x-m3)/s3)**2))
    
    popt,pcov = curve_fit(fun3,WL_interp,p,
                          bounds=([0,0,0,380,500,580,0,0,0],
                                  [1,1,1,500,600,780,50,50,50]))
    print(popt)
    (rx,ry) = chroma_cordinate1931(popt[2]*np.exp(-((WL_interp-popt[5])/popt[8])**2),
                                                      x_lambda,y_lambda,z_lambda)
    (ru,rv) = trans_1931_1976(rx,ry)
    print(ru,rv)
    (gx,gy) = chroma_cordinate1931(popt[1]*np.exp(-((WL_interp-popt[4])/popt[7])**2),
                                                      x_lambda,y_lambda,z_lambda)
    (gu,gv) = trans_1931_1976(gx,gy)
    print(gu,gv)
    (bx,by) = chroma_cordinate1931(popt[0]*np.exp(-((WL_interp-popt[3])/popt[6])**2),
                                                      x_lambda,y_lambda,z_lambda)
    (bu,bv) = trans_1931_1976(bx,by)
    print(bu,bv)
    return(ru,rv,gu,gv,bu,bv)

In [10]:
def rgb_1931(p,x_lambda,y_lambda,z_lambda,WL_interp):
    def fun3(x,a1,a2,a3,m1,m2,m3,s1,s2,s3):
        return (a1*np.exp(-((x-m1)/s1)**2)+
                a2*np.exp(-((x-m2)/s2)**2)+
                a3*np.exp(-((x-m3)/s3)**2))
    
    popt,pcov = curve_fit(fun3,WL_interp,p,
                          bounds=([0,0,0,380,500,580,0,0,0],
                                  [1,1,1,500,600,780,50,50,50]))
    
    (rx,ry) = chroma_cordinate1931(popt[2]*np.exp(-((WL_interp-popt[5])/popt[8])**2),
                                                      x_lambda,y_lambda,z_lambda)
    
    (gx,gy) = chroma_cordinate1931(popt[1]*np.exp(-((WL_interp-popt[4])/popt[7])**2),
                                                      x_lambda,y_lambda,z_lambda)
    
    (bx,by) = chroma_cordinate1931(popt[0]*np.exp(-((WL_interp-popt[3])/popt[6])**2),
                                                      x_lambda,y_lambda,z_lambda)
    return(rx,ry,gx,gy,bx,by)

In [11]:
def V(WL_interp):
    '''visual function'''
    file = path+'V_lambda1.csv'
    data = np.loadtxt(open(file),delimiter = ',')
    WL = data[:,0]
    spec_origin = data[:,1]                                #原始光谱
    func = scipy.interpolate.interp1d(WL,spec_origin,kind='linear')
    V = func(WL_interp)
    return(V)

def C(WL_interp):
    '''circadian function'''
    a = 1.741/(1+np.exp(14.397-WL_interp/30.582))
    b = np.exp(27.885-WL_interp/17.736)
    return(-0.014+a*b/(1+b))

def B(WL_interp):
    '''
    B1 = 6.737*1e-4+0.2361*np.exp(-(WL_interp-416.136)**2/20.276)
    B2 = 0.4443*np.exp(-(WL_interp-423.378)**2/215.925)
    B3 = 0.8606*np.exp(-(WL_interp-447.663)**2/804.406)
    B4 = 0.1505*np.exp(-(WL_interp-480.662)**2/118.811)
    B5 = 0.0908*np.exp(-(WL_interp-471.588)**2/2697.525)
    B = B1+B2+B3+B4+B5'''
    file = path+'B_lambda.csv'
    data = np.loadtxt(open(file),delimiter = ',')
    WL = data[:,0]
    spec_origin = data[:,1]                                #原始光谱
    func = scipy.interpolate.interp1d(WL,spec_origin,kind='linear')
    B = func(WL_interp)
    return(B)

In [12]:
def KB(p,WL_interp):
    return(np.sum(p*B(WL_interp))/np.sum(p*V(WL_interp))/683)

def KC(p,WL_interp):
    return(3616*np.sum(p*B(WL_interp))/np.sum(p*V(WL_interp))/683)

def RB(p):
    blue = np.sum(p[20:121])/np.sum(p)
    return(blue)